<a href="https://colab.research.google.com/github/Kemi41/orbit/blob/main/test_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Creating JSON to Train the Model

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import os

In [ ]:
!wget -O plagcheckfile.csv https://raw.githubusercontent.com/arifanf/MSIB21/main/Dataset/plagcheckfile.csv

--2021-12-08 01:59:16--  https://raw.githubusercontent.com/arifanf/MSIB21/main/Dataset/plagcheckfile.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1667 (1.6K) [text/plain]
Saving to: ‘plagcheckfile.csv’

plagcheckfile.csv   100%[===================>]   1.63K  --.-KB/s    in 0s      

2021-12-08 01:59:16 (17.6 MB/s) - ‘plagcheckfile.csv’ saved [1667/1667]



In [ ]:
PATH = os.path.dirname('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
# a function to clean the texts from the CSV file
def cleanText(Text):
    # Remove new lines within message
    cleanedText = Text.replace('\n',' ').lower()
    # Deal with some weird tokens
    cleanedText = cleanedText.replace("\xc2\xa0", "")
    # Remove punctuation
    cleanedText = re.sub('([,])','', cleanedText)
    # Remove multiple spaces in message
    cleanedText = re.sub(' +',' ', cleanedText)
    cleanedText = cleanedText.encode('ascii', 'ignore').decode('ascii')
    return cleanedText

In [ ]:
# a function to get data from the CSV file
def getData():
    df = pd.read_csv('plagcheckfile.csv')
    listTexts = df['Text'].values.tolist()
    finallist=[]
    print(listTexts)
    for i in range (0,4):
        textDictionary = {"tag":i}
        listTexts_i = cleanText(listTexts[i])
        print(listTexts_i)
        textDictionary.update(texts = listTexts_i)
        finallist.append(textDictionary)
    finalDictionary={"intents":finallist}       
    return finalDictionary

In [ ]:
# save the dictionary as a file
combinedDictionary = dict()
print ('Getting Text Data')
combinedDictionary.update(getData())
print ('Total len of dictionary', len(combinedDictionary))

print ('Saving text data dictionary')
np.save('\\textDictionary.npy', combinedDictionary)

with open('\\file.txt', 'w') as file:
     file.write(json.dumps(combinedDictionary))

Getting Text Data
['Summarizing a journal article is the process of presenting a focused overview of a completed research study that is published in a peer reviewed scholarly source. A journal article summary provides potential readers with a short descriptive commentary giving them some insight into the articles focus. Writing and summarizing a journal article is a common task for college students and research assistants alike. With a little practice you can learn to read the article effectively with an eye for summary plan a successful summary and write it to completion.', 'The first thing you should do is to decide why you need to summarize the article. If thepurpose of the summary is to take notes to later remind yourself about the article you maywant to write a longer summary. However if the purpose of summarizing the article is toinclude it in a paper you are writing the summary should focus on how the articlesrelates specifically to your paper.', 'An article summary is a short f

### Creating the Machine Learning Model

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
!pip install tflearn
import tflearn

     |████████████████████████████████| 107 kB 5.3 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=00c0a7be640dacc4d6d9fd9005702e0982eae998e9f87e29c2c14e1677459ff4
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
stemmer = LancasterStemmer()

In [ ]:
with open("\\file.txt") as file:
    dataset = json.load(file)

In [ ]:
list_words = []
labels = []
docs_x = [] #List of all the question_patterns.
docs_y = [] #List of all the tags for specific Texts.

### Stemming and Tokenization

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
for intent in dataset["intents"]:
    for pattern in intent["texts"]: #Stems the words. Finds the root of the word. Removes extra characters and symbols to find the root word. 
        split_words = nltk.word_tokenize(pattern) #Tokenizes the words. Breakes the words in the places of spaces and returns a list of all the words in it.
        list_words.extend(split_words) #Using instead of looping and adding one word at a time in the list. It just extends the list untill all the words are in it.
        docs_x.append(split_words) #Adding the pattern of words inside docs_x list.
        docs_y.append(intent["tag"]) #For each pattern, it says what Tag it is a part of.

    if intent["tag"] not in labels:
        labels.append(intent["tag"]) #Adds all the tags in the labels list.

list_words = [stemmer.stem(w.lower()) for w in list_words if w != "?"] #Lower cases all the words in the list_words list. 
list_words = sorted(list(set(list_words))) #Makes a set of the words to remove duplicate. This gives us the actual vocabulary size of the intent. Then converts it back to list and sorts it. 

labels = sorted(labels) #Sorts the labels where the tags are stored.

training = [] #contains the bag of words.
output = [] #The output list to choose the right tag for the output.

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = [] #Makes a list of all the words and tells if those words are occuring to find a pattern.
    #We are representing each sentence with a list the length of the amount of words in our models vocabulary. 
    #Each position in the list will represent a word from our vocabulary. 
    #If the position in the list is a 1 then that will mean that the word exists in our sentence, if it is a 0 then the word is nor present. 

    split_words = [stemmer.stem(w.lower()) for w in doc] #Stems all the words inside docs_x list.

    for w in list_words:
        if w in split_words:
            bag.append(1) #we are putting 1 in the bag of words list for the word (already present in the vocabulary list_words) present in the pattern and 0 if it is not.
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1 #We are looking in the labels list to find were the tag is in that list. We are making that value 1 in the output row. 
    #We will create output lists which are the length of the amount of labels/tags we have in our dataset. 
    #Each position in the list will represent one distinct label/tag, a 1 in any of those positions will show which label/tag is represented.

    training.append(bag) #We are putting the bag of words in the training list. 
    output.append(output_row) #We are putting the output_row list in the output list.

### Bag of words

In [ ]:
def bag_of_words(s, list_words): #bag_of_words function will transform our string input to a bag of words using our created words list
    bag = [0 for _ in range(len(list_words))]

    inp_str_words = nltk.word_tokenize(s)
    inp_str_words = [stemmer.stem(word.lower()) for word in inp_str_words]

    for search_element in inp_str_words:
        for i, w in enumerate(list_words):
            if w == search_element:
                bag[i] = 1 
            
    return numpy.array(bag)

### Building the neural network

In [ ]:
training = np.array(training) #we will convert our training data and output to numpy arrays.
output = np.array(output)
#tensorflow.reset_default_graph() #Resetting all the previous stuffs in the graph.

net = tflearn.input_data(shape=[None, len(training[0])]) #This finds the input shape that we are expecting for the model. Each training input is gonna be of the same length, so, 0.
net = tflearn.fully_connected(net, 8) #Hidden layer with 8 neurons.
net = tflearn.fully_connected(net, 8) #Another hidden layer with 8 neurons.
net = tflearn.fully_connected(net, 8) #Another hidden layer with 8 neurons.
net = tflearn.fully_connected(net, len(output[0]), activation="softmax") #length will be how many labels (how many tags) we have. Give us probability for each neuron in the network. The higest probability will be the output.
net = tflearn.regression(net)

model = tflearn.DNN(net) #Model gets trained.

try:
    model.predict(PATH + "\\model.tflearn")
    model.load()
except:
    model.fit(training, output, n_epoch=2000, batch_size=8, show_metric=True) #Fitting our data to the model. The number of epochs we set is the amount of times that the model will see the same information while training.
    model.save(PATH + "\\model.tflearn") #we can save it to the file model.tflearn for use in other scripts.

Training Step: 68938  | total loss: 1.35090 | time: 0.077s
| Adam | epoch: 334 | loss: 1.35090 - acc: 0.3662 -- iter: 0056/1650


### Function to Check Texts for Plagiarism

In [ ]:
def check():
     
    print("Type an answer (Enter 'quit' to stop)")
    while True:
        inp = input("You: ")
        inp = inp.lower()
        if inp == "quit":
            break

        results = model.predict([bag_of_words(inp, list_words)]) #we predict the most matching text from the training dataset that matches with the input text.
        results_index = numpy.argmax(results) #we get thr index of the result.
        tag = labels[results_index] #we get the tag of the most matching text with the input text.
        
        for tg in dataset["intents"]:
            if tg['tag'] == tag:
                response = tg['texts']
        print("Most similar text to the input: ")
        print(response) #we show the most matching text with the input text. 
        split_sent_resp = nltk.sent_tokenize(response) #we divide the response text (the most matching text with the input text)
        list_sents=split_sent_resp #we put the sentenses in a different list for future work.

        split_sent_inp = nltk.sent_tokenize(inp) #we split the input text into sentenses
        len_inp=len(split_sent_inp) #number of sentenses in the input text.
        list_sents.extend(split_sent_inp)
        total_len=len(list_sents) #we get the total number of sentenses in input and response text, including the duplicate sentenses.
        set_sents=set(list_sents) #we are making the list a set, so that duplicate sentenses get deleted.
        set_len=len(set_sents)
        len_difference=total_len-set_len #this gives us the number of duplicate sentenses, i.e, the copied or plaigarized sentenses.
        plag_percent=(1-((len_inp-len_difference)/len_inp))*100
        print("The percent of plagarism in the document is: ", plag_percent)

### Checking

In [ ]:
check()